In [ ]:
using _Data, _Math
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2, BenchmarkTools, Random

In [ ]:
@pyimport numpy
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.metrics as metrics

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;
rc_params["axes.spines.right"] = false
rc_params["axes.spines.top"] = false

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")
include("../../../functions/utils.jl")

In [ ]:
analyzer = "chuyu"

In [ ]:
data_info_dict = load("/home/byoungsoo/Notebooks/project_place_cell/figures/chuyu/figure_data_info.jld2")

In [ ]:
keys(data_info_dict)

In [ ]:
job_names_ordered = [
 "corner_cue_rotation_whole"
 "landmark_removal"
 "boundary_morphing"
 "job_retangular_half_circle_rotation"

    
 "hedgehog_rotation_bottom_out"
 "hedgehog_landmark_removal"
 "hedgehog_circle"
 "corner_cue_circle"
    
]


job_names_figure = [
 "Chamber rotation"
 "Landmark removal"
 "Wall morphing"
    
 "Wall rotation"
    
 "No change"
 "Landmark removal"
 "Wall morphing"
 "Wall morphing + \n Landmark removal"
]


In [ ]:
fig_path = "/home/byoungsoo/Notebooks/project_place_cell/figures/output/review_figures/new_dropout/"

In [ ]:
# percentage of area overlap in PF
function PF_overlap(which_map_1, which_map_2)
    PF1 = map_field(which_map_1)
    PF2 = map_field(which_map_2)
    overlap_PF = PF1.*PF2
    union_PF = (PF1.+PF2 .> 0)
    
    size_unionPF = sum(union_PF)
    size_overlapPF = sum(overlap_PF)

    overlap_percentage = size_overlapPF/size_unionPF

    return overlap_percentage
end

function PF_corr(which_map_1, which_map_2)
    PF1 = map_field(which_map_1)
    PF2 = map_field(which_map_2)
    union_PF = (PF1.+PF2 .> 0)

    PF_map_1 = fill(NaN32, size(which_map_1))
    PF_map_1[union_PF] .= which_map_1[union_PF]
    PF_map_2 = fill(NaN32, size(which_map_2))
    PF_map_2[union_PF] .= which_map_2[union_PF]

    return corr_2d_original(PF_map_1,PF_map_2)
end

In [ ]:
morphing_need_data = [data_info_dict["boundary_morphing"];data_info_dict["hedgehog_circle"];data_info_dict["corner_cue_circle"]]
morphing_need_filename = [data_info[3] for data_info in morphing_need_data];

In [ ]:
filepath = "/home/byoungsoo/Notebooks/project_place_cell/figures/byoungsoo/dropout"
fwd_data = load(joinpath(filepath, "place_cell_dropout.jld"))

In [ ]:
# plot place cell drop out
fig = figure(figsize=(7,1.5))
tick_locs = []
tick_labels = []
overlap_thr = 0.1
overlap_thr_2 = 0.1

flag = 1
temp_flag = 0
job_flag = 1
other_flag = 19
for (i_job, job_name) in enumerate(job_names_ordered)
    data_info_all = data_info_dict[job_name]
    job_name = job_names_figure[i_job]

    job_length = length(data_info_all)
    if job_name == "Landmark removal" && job_flag == 0
        temp_flag = flag
        flag = other_flag
    
    println(job_name)
    println(job_length)

    earlylate_place_cell_1 = fwd_data["earlylate_place_cell_1_tog"][flag:flag+job_length-1]
    earlylate_place_cell_2 = fwd_data["earlylate_place_cell_2_tog"][flag:flag+job_length-1]
    
    trials_place_cell_1 = fwd_data["trials_place_cell_1_tog"][flag:flag+job_length-1]
    trials_place_cell_2 = fwd_data["trials_place_cell_2_tog"][flag:flag+job_length-1]

    fwd_earlylate_overlapPF = fwd_data["earlylate_PFoverlap_tog"][flag:flag+job_length-1]
    fwd_trials_overlapPF = fwd_data["trials_PFoverlap_tog"][flag:flag+job_length-1]

    fwd_earlylate_pvals = fwd_data["earlylate_one_sided_p_values_tog"][flag:flag+job_length-1]
    fwd_trials_pvals = fwd_data["trials_one_sided_p_values_tog"][flag:flag+job_length-1]

    if job_name == "Landmark removal" && job_flag == 0
        flag = temp_flag+job_length-1
    else
        flag = flag+job_length
    end

    if job_name == "Landmark removal" && job_flag == 1
        job_flag = 0
    end
    
    plot_y_all = []
    i_plot = i_job
    for i in 1:length(fwd_earlylate_overlapPF)
        earlylate_place_cell_1_index = earlylate_place_cell_1[i]
        earlylate_place_cell_2_index = earlylate_place_cell_2[i]
        trials_place_cell_1_index = trials_place_cell_1[i]
        trials_place_cell_2_index = trials_place_cell_2[i]
        
        fwd_earlylate_non_significant_PF_index = earlylate_place_cell_1_index[fwd_earlylate_pvals[i] .>= 5e-2]
        fwd_trials_non_significant_PF_index = trials_place_cell_1_index[fwd_trials_pvals[i] .>= 5e-2]
        
        fwd_earlylate_PC_at_s1 = earlylate_place_cell_1_index
        fwd_earlylate_non_PC_at_s2 = setdiff(collect(1:length(fwd_earlylate_overlapPF[i])),earlylate_place_cell_2_index)
        fwd_earlylate_non_PC_at_s2 = intersect(fwd_earlylate_non_PC_at_s2,fwd_earlylate_non_significant_PF_index)
        
        fwd_trials_PC_at_s1 = trials_place_cell_1_index
        fwd_trials_non_PC_at_s2 = setdiff(collect(1:length(fwd_trials_overlapPF[i])),trials_place_cell_2_index)
        fwd_trials_non_PC_at_s2 = intersect(fwd_trials_non_PC_at_s2,fwd_trials_non_significant_PF_index)
        
        fwd_earlylate_overlapping_place_cell = [which_neuron for which_neuron in fwd_earlylate_PC_at_s1 if !(which_neuron in intersect(fwd_earlylate_PC_at_s1,fwd_earlylate_non_PC_at_s2))];
        fwd_trials_overlapping_place_cell = [which_neuron for which_neuron in fwd_trials_PC_at_s1 if !(which_neuron in intersect(fwd_trials_PC_at_s1,fwd_trials_non_PC_at_s2))];

        earlylate_PC_at_1st = fwd_earlylate_PC_at_s1
        trials_PC_at_1st = fwd_trials_PC_at_s1

        earlylate_dropout_PC = fwd_earlylate_non_PC_at_s2
        trials_dropout_PC = fwd_trials_non_PC_at_s2

        if i_job>4
                i_plot = i_job+1
        end
        
        plot([i_plot*2-1,i_plot*2], [100*length(earlylate_dropout_PC)/length(earlylate_PC_at_1st),100*length(trials_dropout_PC)/length(trials_PC_at_1st)], color="gray")
        scatter([i_plot*2-1], [100*length(earlylate_dropout_PC)/length(earlylate_PC_at_1st)], color="gray", alpha=0.7)
        scatter([i_plot*2], [100*length(trials_dropout_PC)/length(trials_PC_at_1st)], color="r", alpha=0.7)
    end
    append!(tick_locs, [i_plot*2-0.5])
    append!(tick_labels, [job_name])
end

xticks(tick_locs, tick_labels, rotation=45)
ylabel("Dropout PCs (%)")
ylim(-5,80)
fig.savefig(joinpath(fig_path, "dropout_pc_fwd.pdf"), bbox_inches="tight",transparent = true)

In [ ]:
# PF correlation vs. PF overlap, s1 => s2
job_stack = 1
job_flag = 0
i_job = 1
job_name = job_names_ordered[i_job]
data_info_this_job = data_info_dict[job_name]
job_name = job_names_figure[i_job]
job_length = length(data_info_this_job)

println(job_name)

fig = figure(figsize=(1,1))
for exp_idx in job_stack:(job_stack+job_length-1)
    fwd_trials_PC = fwd_data["trials_place_cell_1_tog"][exp_idx]
    union_trials_PC = fwd_trials_PC

    fwd_trials_PFcorr = fwd_data["trials_original_PFcor_tog"][exp_idx]
    fwd_trials_PFoverlap = fwd_data["trials_PFoverlap_tog"][exp_idx]
    fwd_trials_PFcov_pval = fwd_data["trials_one_sided_p_values_tog"][exp_idx]
    
    fwd_red_idx = fwd_trials_PC[findall(fwd_trials_PFcov_pval .< 5e-2)]
    union_red_idx = fwd_red_idx
    
    scatter(fwd_trials_PFcorr[setdiff(union_trials_PC,union_red_idx)],fwd_trials_PFoverlap[setdiff(union_trials_PC,union_red_idx)],color="k",s=1,alpha=0.1)
    scatter(fwd_trials_PFcorr[union_red_idx],fwd_trials_PFoverlap[union_red_idx],color="orange",s=1,alpha=0.1)
            
    xlabel("PF cor.")
    ylabel("PF overlap")
    xlim(-1.05,1)
    ylim(-0.05,1)
    fig.savefig(joinpath(fig_path, "trials_PFcor_v_PFoverlap_$(i_job).pdf"), bbox_inches="tight",transparent = true);
end
job_stack = job_stack+job_length
if job_name == "Landmark removal" && job_flag == 0
    job_flag = 1
    job_stack = job_stack - 1
end

In [ ]:
# PF correlation vs. PF overlap, control
job_stack = 1
job_flag = 0
i_job = 1
job_name = job_names_ordered[i_job]
data_info_this_job = data_info_dict[job_name]
job_name = job_names_figure[i_job]
job_length = length(data_info_this_job)

println(job_name)

fig = figure(figsize=(1,1))
for exp_idx in job_stack:(job_stack+job_length-1)
    fwd_earlylate_PC = fwd_data["earlylate_place_cell_1_tog"][exp_idx]
    union_earlylate_PC = fwd_earlylate_PC

    fwd_earlylate_PFcorr = fwd_data["earlylate_original_PFcor_tog"][exp_idx]
    fwd_earlylate_PFoverlap = fwd_data["earlylate_PFoverlap_tog"][exp_idx]
    fwd_earlylate_PFcov_pval = fwd_data["earlylate_one_sided_p_values_tog"][exp_idx]
    
    fwd_red_idx = fwd_earlylate_PC[findall(fwd_earlylate_PFcov_pval .< 5e-2)]
    union_red_idx = fwd_red_idx
    
    scatter(fwd_earlylate_PFcorr[setdiff(union_earlylate_PC,union_red_idx)],fwd_earlylate_PFoverlap[setdiff(union_earlylate_PC,union_red_idx)],color="k",s=1,alpha=0.1)
    scatter(fwd_earlylate_PFcorr[union_red_idx],fwd_earlylate_PFoverlap[union_red_idx],color="orange",s=1,alpha=0.1)
    xlabel("PF cor.")
    ylabel("PF overlap")
    xlim(-1.05,1)
    ylim(-0.05,1)
    fig.savefig(joinpath(fig_path, "earlylate_PFcor_v_PFoverlap_$(i_job).pdf"), bbox_inches="tight",transparent = true);
end
job_stack = job_stack+job_length
if job_name == "Landmark removal" && job_flag == 0
    job_flag = 1
    job_stack = job_stack - 1
end